In [2]:
import numpy as np

In [3]:
# Load the data
data = np.loadtxt("databp.csv", delimiter=',', skiprows=0)  # skiprows=0 to not skip the header
print(data)

[[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 2.]
 [1. 0.]
 [3. 2.]
 [0. 0.]
 [3. 1.]
 [1. 1.]
 [1. 1.]
 [3. 0.]
 [3. 1.]
 [3. 0.]
 [1. 0.]
 [0. 0.]
 [0. 2.]
 [2. 2.]
 [3. 2.]
 [2. 0.]
 [2. 1.]
 [0. 1.]
 [1. 2.]
 [1. 3.]
 [0. 4.]
 [0. 3.]
 [0. 0.]
 [0. 2.]
 [1. 2.]
 [2. 1.]
 [3. 2.]
 [3. 2.]
 [3. 1.]
 [1. 1.]
 [1. 1.]
 [0. 0.]
 [1. 0.]
 [1. 1.]
 [2. 1.]
 [2. 1.]
 [0. 2.]
 [0. 1.]
 [0. 2.]
 [0. 2.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 2.]
 [1. 2.]
 [1. 1.]
 [0. 3.]
 [0. 0.]
 [0. 1.]
 [0. 1.]
 [0. 0.]
 [0. 2.]
 [1. 2.]
 [1. 1.]
 [0. 1.]
 [0. 0.]
 [0. 2.]
 [0. 0.]
 [1. 0.]
 [1. 1.]
 [0. 1.]
 [0. 2.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 0.]
 [1. 0.]
 [2. 1.]
 [2. 1.]
 [2. 3.]
 [3. 0.]
 [3. 1.]
 [0. 0.]
 [1. 1.]
 [2. 1.]
 [4. 1.]
 [4. 1.]
 [1. 2.]
 [3. 3.]
 [1. 2.]
 [0. 1.]
 [0. 2.]
 [1. 3.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [2. 1.]
 [3. 1.]
 [0. 1.]
 [0. 4.]
 [1. 1.]
 [0. 5.]
 [0. 3.]
 [0. 2.]
 [0. 1.]
 [1. 0.]
 [2. 1.]
 [0. 1.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 

In [4]:
# Center the data
def center_data(data):
    means = data.mean(axis=0)
    Y = data - means
    return Y

In [5]:
centered_data = center_data(data)
print(centered_data)

[[-0.866 -1.556]
 [-0.866 -0.556]
 [ 0.134 -1.556]
 [ 0.134  0.444]
 [ 0.134 -1.556]
 [ 2.134  0.444]
 [-0.866 -1.556]
 [ 2.134 -0.556]
 [ 0.134 -0.556]
 [ 0.134 -0.556]
 [ 2.134 -1.556]
 [ 2.134 -0.556]
 [ 2.134 -1.556]
 [ 0.134 -1.556]
 [-0.866 -1.556]
 [-0.866  0.444]
 [ 1.134  0.444]
 [ 2.134  0.444]
 [ 1.134 -1.556]
 [ 1.134 -0.556]
 [-0.866 -0.556]
 [ 0.134  0.444]
 [ 0.134  1.444]
 [-0.866  2.444]
 [-0.866  1.444]
 [-0.866 -1.556]
 [-0.866  0.444]
 [ 0.134  0.444]
 [ 1.134 -0.556]
 [ 2.134  0.444]
 [ 2.134  0.444]
 [ 2.134 -0.556]
 [ 0.134 -0.556]
 [ 0.134 -0.556]
 [-0.866 -1.556]
 [ 0.134 -1.556]
 [ 0.134 -0.556]
 [ 1.134 -0.556]
 [ 1.134 -0.556]
 [-0.866  0.444]
 [-0.866 -0.556]
 [-0.866  0.444]
 [-0.866  0.444]
 [-0.866 -1.556]
 [-0.866 -0.556]
 [-0.866 -1.556]
 [ 0.134 -1.556]
 [-0.866 -1.556]
 [-0.866 -1.556]
 [-0.866 -1.556]
 [-0.866  0.444]
 [ 0.134  0.444]
 [ 0.134 -0.556]
 [-0.866  1.444]
 [-0.866 -1.556]
 [-0.866 -0.556]
 [-0.866 -0.556]
 [-0.866 -1.556]
 [-0.866  0.44

In [6]:
# Fit VAR model
def fit_VAR(data, p):
    n, k = data.shape
    X = center_data(data)
    
    # Construct the regressor matrix Xprime
    Xprime = np.zeros((n - p, k * p))
    X_target = np.zeros((n - p, k))
    
    for t in range(p, n):
        Xprime_row = []
        for lag in range(1, p + 1):
            Xprime_row.extend(X[t - lag])
        Xprime[t - p, :] = Xprime_row
        X_target[t - p] = X[t]
    
    # Estimate coefficients using OLS
    beta = np.linalg.inv(Xprime.T @ Xprime) @ Xprime.T @ X_target
    # Predicting X_hat
    X_hat = Xprime @ beta
    residuals = X[p:] - X_hat
    
    return residuals, beta

In [7]:
residuals, beta = fit_VAR(data, 3)
print(beta)

# changement de forme pour adapter et comparer a l'output de la fonction de fit statsmodels
beta_reshaped = beta.T.reshape(2, 3, 2)
print(beta_reshaped)

[[ 0.57420665  0.03814606]
 [-0.02487775  0.26593181]
 [-0.02069212  0.0594241 ]
 [ 0.02640971  0.25846528]
 [ 0.07995184 -0.0409468 ]
 [ 0.02792318  0.06363696]]
[[[ 0.57420665 -0.02487775]
  [-0.02069212  0.02640971]
  [ 0.07995184  0.02792318]]

 [[ 0.03814606  0.26593181]
  [ 0.0594241   0.25846528]
  [-0.0409468   0.06363696]]]


In [8]:
x1 = np.random.rand(5,1)
print(x1)
x2 = np.random.rand(5,1)
print(x2)
xc = np.concatenate((x1, x2), axis = 1)
print(xc, np.shape(xc))
xc[3,:] = xc[1,0], xc[2,0]
print(xc[3:,0])


[[0.32514028]
 [0.59745533]
 [0.23169089]
 [0.0109393 ]
 [0.51107997]]
[[0.20949875]
 [0.26208052]
 [0.51979602]
 [0.64102952]
 [0.57917436]]
[[0.32514028 0.20949875]
 [0.59745533 0.26208052]
 [0.23169089 0.51979602]
 [0.0109393  0.64102952]
 [0.51107997 0.57917436]] (5, 2)
[0.59745533 0.51107997]


In [23]:
n = 497
print(data)
X1 = data[p+1:, 0]
X2 = data[p:, 1]
#Xs = np.zeros((n,2))
#Xs[:,0] = X1
#Xs[:,1] = X2
#print(f"xs:{Xs}")
print(np.shape(X1))

[[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 2.]
 [1. 0.]
 [3. 2.]
 [0. 0.]
 [3. 1.]
 [1. 1.]
 [1. 1.]
 [3. 0.]
 [3. 1.]
 [3. 0.]
 [1. 0.]
 [0. 0.]
 [0. 2.]
 [2. 2.]
 [3. 2.]
 [2. 0.]
 [2. 1.]
 [0. 1.]
 [1. 2.]
 [1. 3.]
 [0. 4.]
 [0. 3.]
 [0. 0.]
 [0. 2.]
 [1. 2.]
 [2. 1.]
 [3. 2.]
 [3. 2.]
 [3. 1.]
 [1. 1.]
 [1. 1.]
 [0. 0.]
 [1. 0.]
 [1. 1.]
 [2. 1.]
 [2. 1.]
 [0. 2.]
 [0. 1.]
 [0. 2.]
 [0. 2.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 2.]
 [1. 2.]
 [1. 1.]
 [0. 3.]
 [0. 0.]
 [0. 1.]
 [0. 1.]
 [0. 0.]
 [0. 2.]
 [1. 2.]
 [1. 1.]
 [0. 1.]
 [0. 0.]
 [0. 2.]
 [0. 0.]
 [1. 0.]
 [1. 1.]
 [0. 1.]
 [0. 2.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 0.]
 [1. 0.]
 [2. 1.]
 [2. 1.]
 [2. 3.]
 [3. 0.]
 [3. 1.]
 [0. 0.]
 [1. 1.]
 [2. 1.]
 [4. 1.]
 [4. 1.]
 [1. 2.]
 [3. 3.]
 [1. 2.]
 [0. 1.]
 [0. 2.]
 [1. 3.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [2. 1.]
 [3. 1.]
 [0. 1.]
 [0. 4.]
 [1. 1.]
 [0. 5.]
 [0. 3.]
 [0. 2.]
 [0. 1.]
 [1. 0.]
 [2. 1.]
 [0. 1.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 

In [18]:
print(residuals)

[[ 0.12779772  1.11140548]
 [ 0.19673186 -1.28505393]
 [ 2.04212829  0.8344629 ]
 [-2.05955647 -1.38403622]
 [ 2.65771917 -0.24623924]
 [-1.26503001  0.02321329]
 [ 0.21475181 -0.33279864]
 [ 1.90558889 -1.154747  ]
 [ 0.89220151 -0.04700091]
 [ 0.9848732  -1.17331565]
 [-1.19839475 -1.02031855]
 [-1.05149492 -0.74919811]
 [-0.49074932  1.47143311]
 [ 1.69821774  0.9171    ]
 [ 1.57693685  0.28293052]
 [-0.01173193 -2.00133767]
 [ 0.37350767 -0.40885849]
 [-1.64943912 -0.05748885]
 [ 0.60836267  0.84666513]
 [-0.01027364  1.59779849]
 [-0.83120981  1.9320837 ]
 [-0.36641014  0.42314251]
 [-0.46631289 -2.57360381]
 [-0.46250776  0.37807313]
 [ 0.69440003  0.68524231]
 [ 1.15114351 -0.67892342]
 [ 1.51690486  0.3621643 ]
 [ 0.93472594  0.2980744 ]
 [ 0.87697969 -0.91523121]
 [-1.25577307 -0.67198956]
 [-0.08095006 -0.33723216]
 [-1.09441111 -1.154747  ]
 [ 0.61482147 -0.93256273]
 [ 0.04633241  0.34718059]
 [ 1.19977729  0.04451484]
 [ 0.51920909 -0.2111497 ]
 [-1.48802197  0.66578924]
 

In [27]:
# Function to estimate VAR-ARMA parameters
def estimate_VAR_ARMA(data, p, residuals): #p = ordre du VAR sur les donnees centrees, necessaire pour les retards et valeurs manquantes des residus
    T, N = data.shape
    assert N == 2, "This implementation is for bivariate VAR only."
    print(f"T = {T}")
    n = T - p  # Number of rows after adjusting for lags
    Xprime = np.zeros((n-1, 5)) #n lignes pour les pbs, 3 = 1 cst + 2 termes retard X_t-1 + 2 terme retard et-1
    Xprime[:, 0] = 1  # colonne de la constante 
    print(f"n = {n}")
    for t in range(0, n-1):
        Xprime[t, 1:3] = data[p+t, :] #pour t= 0 le terme le plus lointain dans nos données (celui de retard 1) est le (p+1)eme element de nos données, et le premier qui a un terme de retard correspondant
        Xprime[t, 3:5] = residuals[t, :]
    print(Xprime)
    X1 = data[p+1:, 0]
    X2 = data[p+1:, 1]
    Xs = np.zeros((n-1,2))
    Xs[:,0] = X1
    Xs[:,1] = X2

    # Estimate coefficients using OLS
    beta = np.linalg.inv(Xprime.T @ Xprime) @ Xprime.T @ Xs

    Phi = np.zeros((2, 2))
    Theta = np.zeros((2, 2))
    omega = [beta[0, 0], beta[0, 1]]
    Theta[0, :] = beta[3,0], beta[4,0]
    Theta[1, :] = beta[3,1], beta[4,1]
    Phi[0, :] = beta[1,0], beta[1,1]
    Phi[1, :] = beta[2,0], beta[2,1]
    
    return omega, Theta, Phi

In [28]:
# Assuming `data` and `residuals` are already defined
p = 3
omega, Theta, Phi = estimate_VAR_ARMA(data, p, residuals)
print("Omega:\n", omega)
print("Theta:\n", Theta)
print("Phi:\n", Phi)

T = 500
n = 497
[[ 1.          1.          2.          0.12779772  1.11140548]
 [ 1.          1.          0.          0.19673186 -1.28505393]
 [ 1.          3.          2.          2.04212829  0.8344629 ]
 ...
 [ 1.          0.          2.         -1.4096021   0.10044535]
 [ 1.          0.          3.         -0.44901565  1.19499411]
 [ 1.          0.          1.         -0.46552213 -0.9520895 ]]
Omega:
 [0.21499747257442814, 0.24155596673041202]
Theta:
 [[-0.04512705 -0.09949254]
 [ 0.01704663 -0.56768375]]
Phi:
 [[0.61645031 0.02440262]
 [0.07607898 0.83447398]]
